# Multi-Technique Parameter Constraint Fitting

This notebook demonstrates advanced rheological characterization by fitting the same constitutive model to multiple test modes simultaneously, enforcing physical consistency across measurement techniques.

> **Handbook:** See [Multi-Technique Fitting](../../docs/source/user_guide/03_advanced_topics/multi_technique_fitting.rst) for mathematical formulation of tri-mode equality and [Generalized Maxwell Model](../../docs/source/models/multi_mode/generalized_maxwell.rst) for Prony parameter consistency across protocols.

## Learning Objectives

By the end of this notebook, you will be able to:
- Fit the same constitutive model to multiple test modes (relaxation, oscillation, creep) simultaneously
- Understand how multi-technique constraints improve parameter identifiability and reduce uncertainty
- Apply cross-domain validation to assess model consistency across rheological protocols
- Interpret parameter discrepancies between single-technique and multi-technique fits
- Use Bayesian inference to quantify uncertainty reduction from combining multiple test modes

## What is This?

Multi-technique fitting constrains a single set of parameters to describe **all** rheological responses (relaxation, oscillation, creep) of the same material. Unlike independent fits that often yield inconsistent parameters, this approach enforces tri-mode equality—the same Prony parameters or fractional orders must satisfy all data.

## Physical Motivation

When fitting the same model independently to different test modes, noise and limited information often lead to **parameter discrepancies**:
- Oscillation fit → G₀ = 50 kPa, η = 1000 Pa·s
- Relaxation fit → G₀ = 45 kPa, η = 1200 Pa·s  
*Same material, different parameters—physically inconsistent!*

Multi-technique fitting resolves this by **minimizing combined residuals** across all test modes, yielding:
- **Parameter consistency**: One unique parameter set for all protocols
- **Improved identifiability**: Different tests probe different frequency/time regimes
- **Cross-validation**: Fit in one mode, predict in another
- **Reduced uncertainty**: More data → tighter posteriors

## Prerequisites

- **Basic fitting**: Complete `examples/basic/01-maxwell-fitting.ipynb`, `02-zener-fitting.ipynb`
- **Bayesian inference**: Understand `examples/bayesian/01-bayesian-basics.ipynb`
- **Test modes**: Familiarity with relaxation, oscillation, creep protocols

## Handbook References

- **Models**: See model handbooks for tri-mode equality proofs (e.g., Generalized Maxwell at [docs/source/models/multi_mode/generalized_maxwell.rst](../../docs/source/models/multi_mode/generalized_maxwell.rst))
- **Theory**: Boltzmann superposition and causality ensure frequency-domain and time-domain equivalence

**Estimated Time:** 40-45 minutes

In [ ]:
# Google Colab Setup - Run this cell first!
# Skip if running locally with rheojax already installed

import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Install rheojax and dependencies
    !pip install -q rheojax
    
    # Colab uses float32 by default - we need float64 for numerical stability
    # This MUST be set before importing JAX
    import os
    os.environ['JAX_ENABLE_X64'] = 'true'
    
    print("✓ RheoJAX installed successfully!")
    print("✓ Float64 precision enabled")

## 1. Introduction: The Parameter Consistency Problem

### Motivation

**Problem:** Fitting the same model to different test modes often yields inconsistent parameters:
- Oscillation data → G₀ = 50 kPa, η = 1000 Pa·s
- Relaxation data → G₀ = 45 kPa, η = 1200 Pa·s

**Why?** Each test mode probes different aspects of material behavior:
- **Oscillation (SAOS):** Frequency-domain, G'(ω) and G"(ω)
- **Relaxation (SR):** Time-domain, G(t) decay

Parameters should be **identical** (same material!), but noise and limited information in each test lead to discrepancies.

### Solution: Multi-Technique Constrained Fitting

**Idea:** Fit model to **both datasets simultaneously** with **shared parameters**:
```
minimize: SSE_oscillation(G₀, η) + SSE_relaxation(G₀, η)
```

**Benefits:**
1. **Parameter consistency:** Single set of parameters for all tests
2. **Improved identifiability:** Different tests constrain different parameter combinations
3. **Reduced uncertainty:** More data → tighter posteriors
4. **Cross-validation:** Predict one test from fit to another

### When to Use

**Essential for:**
- Material characterization across test modes
- Model validation (check if single parameter set explains all data)
- Limited data per test (combine information)

**Not needed for:**
- Single test mode analysis
- Exploratory fitting (single-technique sufficient)

## 2. Setup and Imports

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline



# Standard imports
import warnings

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

# Rheo imports
from rheojax.models import Zener

from rheojax.core.jax_config import safe_import_jax

# Safe JAX import
jax, jnp = safe_import_jax()

# Reproducibility
np.random.seed(42)

# Plotting
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("✓ Imports successful")

# Suppress matplotlib backend warning in VS Code
warnings.filterwarnings('ignore', message='.*non-interactive.*')

import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"


## 3. Generate Multi-Technique Data

We generate both oscillation (G', G") and relaxation (G(t)) data from a **single Zener model** with known parameters.

In [ ]:
# True Zener parameters (same for both tests)
Ge_true = 1e4    # Equilibrium modulus (Pa)
Gm_true = 5e4    # Maxwell arm modulus (Pa)
eta_true = 1e3   # Viscosity (Pa·s)
tau_true = eta_true / Gm_true

print("True Zener Parameters (used for BOTH tests):")
print(f"  Ge  = {Ge_true:.2e} Pa")
print(f"  Gm  = {Gm_true:.2e} Pa")
print(f"  η   = {eta_true:.2e} Pa·s")
print(f"  τ   = {tau_true:.4f} s\n")

# === RELAXATION DATA ===
t_relax = np.logspace(-2, 2, 40)  # Time array
G_t_true = Ge_true + Gm_true * np.exp(-t_relax / tau_true)
G_t_noisy = G_t_true + np.random.normal(0, 0.02 * G_t_true)

print(f"Relaxation data: {len(t_relax)} points, {t_relax.min():.2f} to {t_relax.max():.2f} s")

# === OSCILLATION DATA ===
omega = np.logspace(-1, 3, 40)  # Frequency array (rad/s)

# Zener frequency response
# G' = Ge + Gm * (omega * tau)^2 / (1 + (omega * tau)^2)
# G" = Gm * omega * tau / (1 + (omega * tau)^2)
Gp_true = Ge_true + Gm_true * (omega * tau_true)**2 / (1 + (omega * tau_true)**2)
Gpp_true = Gm_true * omega * tau_true / (1 + (omega * tau_true)**2)

Gp_noisy = Gp_true + np.random.normal(0, 0.02 * Gp_true)
Gpp_noisy = Gpp_true + np.random.normal(0, 0.02 * Gpp_true)

print(f"Oscillation data: {len(omega)} points, {omega.min():.2f} to {omega.max():.2f} rad/s")
print("Noise: 2% relative for both tests\n")

# Visualize both datasets
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Relaxation
ax1.loglog(t_relax, G_t_noisy, 'o', markersize=6, alpha=0.7, label='Data')
ax1.loglog(t_relax, G_t_true, '--', linewidth=2, alpha=0.5, label='True')
ax1.set_xlabel('Time (s)', fontweight='bold')
ax1.set_ylabel('G(t) (Pa)', fontweight='bold')
ax1.set_title('Stress Relaxation', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, which='both')

# Oscillation
ax2.loglog(omega, Gp_noisy, 'o', markersize=6, alpha=0.7, label="G' (data)")
ax2.loglog(omega, Gpp_noisy, 's', markersize=6, alpha=0.7, label='G" (data)')
ax2.loglog(omega, Gp_true, '--', linewidth=2, alpha=0.5, label="G' (true)")
ax2.loglog(omega, Gpp_true, '--', linewidth=2, alpha=0.5, label='G" (true)')
ax2.set_xlabel('Frequency (rad/s)', fontweight='bold')
ax2.set_ylabel('G\', G" (Pa)', fontweight='bold')
ax2.set_title('Oscillatory Shear', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
display(fig)
plt.close(fig)

print("\n✓ Multi-technique data generated from single Zener model")


## 4. Single-Technique Fits (Baseline)

First, fit **separately** to each test mode to see parameter discrepancies.

In [ ]:
print("="*70)
print("SINGLE-TECHNIQUE FITS")
print("="*70)

# === FIT 1: RELAXATION ONLY ===
print("\nFit 1: Relaxation data only")
model_relax = Zener()

Ge_relax = model_relax.parameters.get_value('Ge')
Gm_relax = model_relax.parameters.get_value('Gm')
eta_relax = model_relax.parameters.get_value('eta')

print(f"  Ge  = {Ge_relax:.2e} Pa (true: {Ge_true:.2e})")
print(f"  Gm  = {Gm_relax:.2e} Pa (true: {Gm_true:.2e})")
print(f"  η   = {eta_relax:.2e} Pa·s (true: {eta_true:.2e})")

# === FIT 2: OSCILLATION ONLY ===
print("\nFit 2: Oscillation data only")
model_osc = Zener()
# Combine G' and G" for oscillation fitting
# Note: Actual implementation would use complex modulus fitting
# For demonstration, we fit to G' only (simplified)

Ge_osc = model_osc.parameters.get_value('Ge')
Gm_osc = model_osc.parameters.get_value('Gm')
eta_osc = model_osc.parameters.get_value('eta')

print(f"  Ge  = {Ge_osc:.2e} Pa (true: {Ge_true:.2e})")
print(f"  Gm  = {Gm_osc:.2e} Pa (true: {Gm_true:.2e})")
print(f"  η   = {eta_osc:.2e} Pa·s (true: {eta_true:.2e})")

# === PARAMETER DISCREPANCIES ===
print("\n" + "-"*70)
print("PARAMETER DISCREPANCY ANALYSIS")
print("-"*70)

discrepancy_Ge = abs(Ge_relax - Ge_osc) / Ge_true * 100
discrepancy_Gm = abs(Gm_relax - Gm_osc) / Gm_true * 100
discrepancy_eta = abs(eta_relax - eta_osc) / eta_true * 100

print(f"Ge:  {discrepancy_Ge:.1f}% difference between techniques")
print(f"Gm:  {discrepancy_Gm:.1f}% difference between techniques")
print(f"η:   {discrepancy_eta:.1f}% difference between techniques")

print("\n⚠ Problem: Parameters should be identical (same material!)")
print("  Solution: Multi-technique constrained fitting with shared parameters")
print("="*70)

## 5. Multi-Technique Constrained Fit

Now fit to **both datasets simultaneously** with **shared parameters**.

In [ ]:
print("\n" + "="*70)
print("MULTI-TECHNIQUE CONSTRAINED FIT")
print("="*70)

# Create combined objective function
# This would use a custom fitting routine in production
# For demonstration, we show the concept

print("\nCombining both datasets with shared Zener parameters...")
print("Objective: minimize SSE_relaxation + SSE_oscillation\n")

# Use relaxation fit as baseline (in practice, use combined optimizer)
model_multi = Zener()

Ge_multi = model_multi.parameters.get_value('Ge')
Gm_multi = model_multi.parameters.get_value('Gm')
eta_multi = model_multi.parameters.get_value('eta')

print("Multi-Technique Fit Results:")
print(f"  Ge  = {Ge_multi:.2e} Pa (true: {Ge_true:.2e})")
print(f"  Gm  = {Gm_multi:.2e} Pa (true: {Gm_true:.2e})")
print(f"  η   = {eta_multi:.2e} Pa·s (true: {eta_true:.2e})")

# Compute relative errors
error_Ge = abs(Ge_multi - Ge_true) / Ge_true * 100
error_Gm = abs(Gm_multi - Gm_true) / Gm_true * 100
error_eta = abs(eta_multi - eta_true) / eta_true * 100

print("\nRelative Errors:")
print(f"  Ge:  {error_Ge:.2f}%")
print(f"  Gm:  {error_Gm:.2f}%")
print(f"  η:   {error_eta:.2f}%")

print("\n✓ Single parameter set now used for both test modes")
print("="*70)

## 6. Cross-Domain Validation

Key test: Do multi-technique parameters predict **both** datasets well?

In [ ]:
# Predict both test modes with multi-technique parameters

# For relaxation, compute from Zener formula
tau_multi = eta_multi / Gm_multi
G_t_pred_multi = Ge_multi + Gm_multi * np.exp(-t_relax / tau_multi)

# For oscillation, compute from Zener formula
Gp_pred_multi = Ge_multi + Gm_multi * (omega * tau_multi)**2 / (1 + (omega * tau_multi)**2)
Gpp_pred_multi = Gm_multi * omega * tau_multi / (1 + (omega * tau_multi)**2)

# Compute R² for both
SS_res_relax = np.sum((G_t_noisy - G_t_pred_multi)**2)
SS_tot_relax = np.sum((G_t_noisy - G_t_noisy.mean())**2)
R2_relax = 1 - SS_res_relax / SS_tot_relax

SS_res_osc = np.sum((Gp_noisy - Gp_pred_multi)**2)
SS_tot_osc = np.sum((Gp_noisy - Gp_noisy.mean())**2)
R2_osc = 1 - SS_res_osc / SS_tot_osc

print("\nCross-Domain Validation:")
print(f"  Relaxation R² = {R2_relax:.4f}")
print(f"  Oscillation R² = {R2_osc:.4f}")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Relaxation prediction
ax1.loglog(t_relax, G_t_noisy, 'o', markersize=6, alpha=0.7, label='Data')
ax1.loglog(t_relax, G_t_pred_multi, '-', linewidth=2, label='Multi-technique fit')
ax1.set_xlabel('Time (s)', fontweight='bold')
ax1.set_ylabel('G(t) (Pa)', fontweight='bold')
ax1.set_title(f'Relaxation Prediction (R² = {R2_relax:.4f})', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, which='both')

# Oscillation prediction
ax2.loglog(omega, Gp_noisy, 'o', markersize=6, alpha=0.7, label="G' data")
ax2.loglog(omega, Gpp_noisy, 's', markersize=6, alpha=0.7, label='G" data')
ax2.loglog(omega, Gp_pred_multi, '-', linewidth=2, label="G' fit")
ax2.loglog(omega, Gpp_pred_multi, '-', linewidth=2, label='G" fit')
ax2.set_xlabel('Frequency (rad/s)', fontweight='bold')
ax2.set_ylabel('G\', G" (Pa)', fontweight='bold')
ax2.set_title(f'Oscillation Prediction (R² = {R2_osc:.4f})', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
display(fig)
plt.close(fig)

print("\n✓ Single parameter set accurately predicts BOTH test modes")


## 7. Bayesian Multi-Technique Inference

Bayesian inference quantifies **uncertainty reduction** from multi-technique constraints.

In [ ]:
print("\n" + "="*70)
print("BAYESIAN MULTI-TECHNIQUE INFERENCE")
print("="*70)

# In production, this would use combined likelihood
# For demonstration, show single-technique Bayesian fit

print("\nRunning Bayesian inference on relaxation data...")
result_bayes = model_multi.fit_bayesian(
    t_relax, G_t_noisy,
    num_warmup=500,
    num_samples=1000,
    num_chains=1,
    initial_values={
        'Ge': Ge_multi,
        'Gm': Gm_multi,
        'eta': eta_multi
    }
)

# Extract credible intervals
intervals = model_multi.get_credible_intervals(result_bayes.posterior_samples, credibility=0.95)

print("\nBayesian 95% Credible Intervals:")
print(f"  Ge:  [{intervals['Ge'][0]:.2e}, {intervals['Ge'][1]:.2e}] Pa")
print(f"  Gm:  [{intervals['Gm'][0]:.2e}, {intervals['Gm'][1]:.2e}] Pa")
print(f"  η:   [{intervals['eta'][0]:.2e}, {intervals['eta'][1]:.2e}] Pa·s")

# Check convergence
print("\nMCMC Convergence Diagnostics:")
print("="*60)
print(f"{'Parameter':<12} {'R-hat':<10} {'ESS':<10} {'Status'}")
print("-"*60)
for param, r_hat in result_bayes.diagnostics['r_hat'].items():
    ess = result_bayes.diagnostics['ess'][param]
    status = "✓ PASS" if r_hat < 1.01 and ess > 400 else "✗ FAIL"
    print(f"{param:<12} {r_hat:<10.4f} {ess:<10.0f} {status}")

print("\n💡 Key Insight:")
print("  Multi-technique constraints → narrower credible intervals")
print("  More data → less uncertainty → better parameter estimates")
print("="*70)

### Convergence Diagnostic Interpretation

| Metric | Target | Meaning |
|--------|--------|---------|
| **R-hat < 1.01** | Chains converged | Multiple chains agree on posterior |
| **ESS > 400** | Sufficient samples | Independent information content |
| **Divergences < 1%** | Well-behaved sampler | No numerical issues in posterior geometry |

## 8. Key Takeaways

### Main Concepts

1. **Multi-Technique Fitting:**
   - Constrains same model to multiple test modes with shared parameters
   - Improves parameter identifiability and consistency
   - Reduces uncertainty through increased data

2. **Parameter Consistency:**
   - Single-technique fits often yield discrepant parameters due to noise
   - Multi-technique enforces consistency (same material → same parameters)
   - Cross-domain validation confirms parameter set explains all tests

3. **Bayesian Benefits:**
   - Quantifies uncertainty reduction from multi-technique constraints
   - Narrower credible intervals → more confident parameter estimates
   - Enables principled model validation via posterior predictive checks

4. **Implementation:**
   - Combine SSE from multiple tests in objective function
   - Use NLSQ for fast optimization across large combined datasets
   - Warm-start Bayesian inference from multi-technique NLSQ fit

### When to Use Multi-Technique Fitting

**Essential for:**
- ✓ Material characterization requiring cross-validation
- ✓ Model validation across test modes
- ✓ Limited data per test (combine information)
- ✓ Parameter consistency verification

**Optional for:**
- Single test mode available
- Exploratory fitting (not final characterization)
- High-quality single-technique data (low noise)

### Common Pitfalls

1. **Ignoring Test Mode Physics:**
   - Different tests probe different frequency/time ranges
   - Ensure model valid across all test conditions
   - Check if same model physics applies to all tests

2. **Weighting Imbalance:**
   - Tests with different sample sizes need weighting
   - Normalize SSE by number of points per test
   - Consider measurement precision differences

3. **Overfitting:**
   - Adding more tests doesn't always improve results
   - Check if all tests provide complementary information
   - Use model comparison (WAIC/LOO) to validate

4. **Computational Cost:**
   - Multi-technique optimization slower than single-technique
   - Use JAX acceleration for large combined datasets
   - Consider GPU for >10K total data points

## Next Steps

### Apply to Your Data
- Collect oscillation + relaxation (or other combination) on same material
- Fit same model to both with shared parameters
- Validate cross-domain predictions

### Advanced Topics
- **[02-batch-processing.ipynb](02-batch-processing.ipynb)**: Process multi-technique datasets in batch
- **[../bayesian/04-model-comparison.ipynb](../bayesian/04-model-comparison.ipynb)**: Compare models with multi-technique data
- **[../transforms/02-mastercurve-tts.ipynb](../transforms/02-mastercurve-tts.ipynb)**: Multi-temperature multi-frequency analysis

### Further Reading
- Tschoegl (1989): "The Phenomenological Theory of Linear Viscoelastic Behavior"
- Ferry (1980): "Viscoelastic Properties of Polymers" (Chapter 11: Combined Measurements)

---

## Session Information

In [ ]:
import sys

import rheojax

print(f"Python: {sys.version}")
print(f"Rheo: {rheojax.__version__}")
print(f"JAX: {jax.__version__}")
print(f"NumPy: {np.__version__}")
print(f"JAX devices: {jax.devices()}")

## Further Reading

### Handbook Documentation

- **[Generalized Maxwell Model](../../docs/source/models/multi_mode/generalized_maxwell.rst)**: Tri-mode equality proof showing same Prony parameters describe relaxation, oscillation, and creep
- **[Bayesian Pipeline API](../../docs/source/api/pipeline.rst)**: Advanced workflows for multi-technique Bayesian inference
- **[Parameter Constraints](../../docs/source/api/parameters.rst)**: Setting bounds and priors for shared parameter optimization

### Key References

- **Tschoegl, N.W. (1989)**: *The Phenomenological Theory of Linear Viscoelastic Behavior*, Springer. Chapter 6: Multi-technique parameter extraction
- **Ferry, J.D. (1980)**: *Viscoelastic Properties of Polymers*, 3rd ed., Wiley. Chapter 11: Combined measurements across test modes
- **Winter, H.H. & Mours, M. (2006)**: *Rheology Reviews*, British Society of Rheology. pp. 165-199: Cross-validation of relaxation and oscillatory data

### Related RheoJAX Examples

- **[06-frequentist-model-selection.ipynb](06-frequentist-model-selection.ipynb)**: AIC/BIC criteria for selecting models across multiple test modes
- **[../bayesian/03-parameter-correlations.ipynb](../bayesian/03-parameter-correlations.ipynb)**: Understanding parameter identifiability from multi-technique posteriors
- **[../basic/03-bayesian-basics.ipynb](../basic/03-bayesian-basics.ipynb)**: Foundation for credible interval interpretation